# HashiCorp Vault Demo for Secrets Sync

HashiCorp Vault's Secrets Sync feature manages the entire lifecycle of the secret and the secret is synchronized to the desired destination provider.  Applications will still be able to natively integrate with the desired destination provider of choice.  This provides an easy way for applications to get the Zero Trust benefits of Vault without having to directly integrate with Vault.  i.e. Centralized secret inventory, audit and reporting, and automation of credential rotation.

Supported destinations include
- AWS Secrets Manager
- Azure Key Vault
- GCP Secret Manager
- GitHub Repository Actions
- Vercel Projects
Ref: https://developer.hashicorp.com/vault/docs/sync#destinations

<img src="images/vault-demo-secrets-sync.png">

In this demo, we will show how key/value secrets that are stored in Vault's KV v2 Secret Engine are actively synchronized into cloud native secret providers on AWS, Azure, and GCP. As a secret is rotated in Vault, the secret is updated to the cloud native secret provider as well.  If the secret is deleted from Vault, it is deleted on the cloud native secret provider as well. This process is asynchronous and event-based. Vault propagates modifications into the proper destinations automatically in a few seconds.

To run this notebook in VS Code, chose the Jupyter kernel and then Bash

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

This also assumes you have the Vault CLI installed.

You will need Vault to be installed with an Enterprise license.

# Setting Up the Vault

In [ ]:
# For this demo, we will be passing our doormat credentials as environment variables.
# For non-doormat scenarios, use your configured IAM programmatic credentials in the secret sync configuration later on.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)

# Refresh Vault docker image with latest version
#docker pull hashicorp/vault-enterprise

# Run Vault in docker in Dev mode with Enterprise license.
# We have set VAULT_LOG_LEVEL to trace for troubleshooting purposes.  This will allow you to view detailed information as you test.
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-e "VAULT_LOG_LEVEL=trace" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:1.15-ent
# Note: Pegging to 1.15-ent as 1.16.1 now does not support AWS session tokens.

In [ ]:
# Verify that Vault is running
docker ps

# Set up a Secret Value in Vault

In [ ]:
# Enable KV v2 secret engine.  We will be storing a secret here.
# This will be used to show the sync to the external secrets engine.
export KV_V2_PATH=demo-kv
vault secrets enable -path=$KV_V2_PATH kv-v2

In [ ]:
# Simulate storing a database secret in the KV v2 secret engine
# We will be generating random values for the userid and password
export MY_SECRET_NAME=demo-database-secret
echo "MY_SECRET_NAME: $MY_SECRET_NAME"
echo
vault kv put -mount=$KV_V2_PATH $MY_SECRET_NAME \
  userid="svc-account-$(openssl rand -base64 10 | tr -dc 'a-zA-Z0-9')" \
  password="$(openssl rand -base64 20 | tr -dc 'a-zA-Z0-9')"

# Secrets Sync with AWS Secrets Manager

Ref: https://developer.hashicorp.com/vault/docs/sync/awssm

Follow the steps to create new IAM user with programmatic keys and the required policy.

Ref: https://developer.hashicorp.com/vault/docs/sync/awssm#setup

In [ ]:
# For HashiCorp staff only.  Log in via doormat and populate your AWS credentials into your environment variables.
doormat login -f && eval $(doormat aws export --role $(doormat aws list | grep -m 1 role))

# For normal usage, you will setup the AWS IAM programmatic credentials that contain the permissions to secrets manager
# Ref: https://developer.hashicorp.com/vault/docs/sync/awssm#setup

# Make sure the AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and AWS_SESSION_TOKEN environment variables are populated
echo
echo "AWS_ACCESS_KEY_ID: ${AWS_ACCESS_KEY_ID}"
echo "AWS_SECRET_ACCESS_KEY: ${AWS_SECRET_ACCESS_KEY}"
echo "AWS_SESSION_TOKEN: ${AWS_SESSION_TOKEN}"

In [ ]:
export AWS_REGION=us-east-1
# Write the configuration for the sync settings
# Note: We are using session_token as our doormat account only allows short-lived credentials.
#       session_token only works in 1.15-ent.  In GA for 1.16.x or later, this is no longer allowed.
#       You will need to use the other command below with a proper IAM programmatic credential.
vault write sys/sync/destinations/aws-sm/my-awssm-1 \
     access_key_id="$AWS_ACCESS_KEY_ID" \
     secret_access_key="$AWS_SECRET_ACCESS_KEY" \
     session_token="$AWS_SESSION_TOKEN" \
     region="$AWS_REGION"

# Use this command when you have the AWS IAM programmatic keys without the session token
# vault write sys/sync/destinations/aws-sm/my-awssm-1 \
#      access_key_id="$AWS_ACCESS_KEY_ID" \
#      secret_access_key="$AWS_SECRET_ACCESS_KEY" \
#      region="$AWS_REGION"


In [ ]:
# Configure the sync to the secret
echo "KV v2 Path: $KV_V2_PATH"
echo "Secret name: $MY_SECRET_NAME"
vault write sys/sync/destinations/aws-sm/my-awssm-1/associations/set \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"

# Check in your AWS Console that the secret has been created
# verify that the secret values match
vault kv get $KV_V2_PATH/$MY_SECRET_NAME

# Secrets Sync with Azure Key Vault

Ref: https://developer.hashicorp.com/vault/docs/sync/azurekv

## Setup your Azure Key Vault resource
1) Create a new "key vault" resource.  Give a name, region, and resource group.  Review and Create.
2) Create a application under App registrations in Microsoft Entra ID to be used by the sync process. Ref: https://learn.microsoft.com/en-us/entra/identity-platform/howto-create-service-principal-portal.  Copy the Application ID (or CLIENT_ID).
3) Under the application, go to Certificates & secrets and create a new client secret for the application and copy the client secret (or CLIENT_SECRET)
4) Assign permissions to key vault to the created application.  Go to the key vault resource > Access control (IAM) > Add role assignment. Select the "Key Vault Secrets Officer" role and select the application. Ref: https://learn.microsoft.com/en-us/azure/key-vault/general/rbac-guide
5) For the account you are using to access the Azure Console, give the "Key Vault Administrator" role so that you can view the synced secret in the later steps.

In [ ]:
# Please Populate the following environment variables with your own settings
export TENANT_ID=
export KEY_VAULT_URI=https://<your vault>.vault.azure.net/
export CLIENT_SECRET=
export CLIENT_ID=

In [ ]:
# Set up the credentials for the sync to Azure Key Vault
echo "Key Vault URI: $KEY_VAULT_URI"
echo "Tenant ID: $TENANT_ID"
echo "Subscription ID: $SUBSCRIPTION_ID"
echo $CLIENT_ID
#echo $CLIENT_SECRET
vault write sys/sync/destinations/azure-kv/my-azure-1 \
    key_vault_uri="$KEY_VAULT_URI" \
    client_id="$CLIENT_ID" \
    client_secret="$CLIENT_SECRET" \
    tenant_id="$TENANT_ID" 
    #subscription_id="$SUBSCRIPTION_ID"

In [ ]:
# Configure the sync to the secret
echo "KV v2 Path: $KV_V2_PATH"
echo "Secret name: $MY_SECRET_NAME"
vault write sys/sync/destinations/azure-kv/my-azure-1/associations/set \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"

# Check in your Azure Console that the secret has been created in Key Vault
# verify that the secret values match
vault kv get $KV_V2_PATH/$MY_SECRET_NAME

# Secrets sync with Google Cloud Platform
Ref:
- https://developer.hashicorp.com/vault/docs/sync/gcpsm

Set your google credentials before running this
- Ref: https://developer.hashicorp.com/vault/docs/sync/gcpsm#provision-service-account
- Ref: https://cloud.google.com/iam/docs/service-accounts-create
1) Enable IAM
2) Create Service Account - IAM & Admin > Service Accounts.  Click "CREATE SERVICE ACCOUNT" and give a name.  Copy the email of the created service account.
3) Click into service account, click on KEYS tab > ADD KEY > Create new key.  Select JSON for key type and click CREATE.
4) Download the json file to this folder and rename to gcp-sm-sync-service-account-credential.json
5) Give permissions to the service account - IAM & Admin > IAM.  Click on "GRANT ACCESS".  Paste the copied email for principals and assign the Secrets Manager Admin role.


In [ ]:
# Make sure you update the credentials json with your own credentials
vault write sys/sync/destinations/gcp-sm/my-dest \
    credentials="@gcp-sm-sync-service-account-credential.json"

In [ ]:
# Configure the sync to the secret
echo "KV v2 Path: $KV_V2_PATH"
echo "Secret name: $MY_SECRET_NAME"
vault write sys/sync/destinations/gcp-sm/my-dest/associations/set \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"

# Check in your GCP Console that the secret has been created in GCP Secret Manager
# verify that the secret values match
vault kv get $KV_V2_PATH/$MY_SECRET_NAME

# Rotating the Secret and Syncing in Action

In [ ]:
# Update the secret value
vault kv put -mount=$KV_V2_PATH $MY_SECRET_NAME \
  userid="svc-account-$(openssl rand -base64 10 | tr -dc 'a-zA-Z0-9')" \
  password="$(openssl rand -base64 20 | tr -dc 'a-zA-Z0-9')"

# Verify that the secret values match
vault kv get $KV_V2_PATH/$MY_SECRET_NAME

# Log into your configured CSP secrets manager and view the new secret value
# For Azure and GCP, refresh to see a new version of the secret

# Cleanup

In [ ]:
# Cleanup AWS Secrets Manager

# Remove association
vault write sys/sync/destinations/aws-sm/my-awssm-1/associations/remove \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"
# Delete secret sync destination
vault delete sys/sync/destinations/aws-sm/my-awssm-1

In [ ]:
# Clean Azure Key Vault
vault write sys/sync/destinations/azure-kv/my-azure-1/associations/remove \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"
# Delete secret sync destination
vault delete sys/sync/destinations/azure-kv/my-azure-1

In [ ]:
# Clean GCP Secrets Manager
vault write sys/sync/destinations/gcp-sm/my-dest/associations/remove \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"
# Delete secret sync destination
vault delete sys/sync/destinations/gcp-sm/my-dest

In [ ]:
# Stop Vault container
docker stop vault-enterprise